# Исследование опроса клиентов телекомунникационной компании

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности (NPS - от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, компания провела опрос, в котором клиентам задавали классический вопрос: «*Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым*». Дашборд и презентацию oб уровне NPS нам и предстоит подготовить.

Импортируем необходимые для работы инструменты:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Подключаемся к базе:

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

В документации к данным указана следующая структура:

- `user_id` - Идентификатор клиента;
- `lt_day` - Количество дней «жизни» клиента;
- `is_new` - Поле хранит информацию о том, является ли клиент новым (новый = лайфтайм не более 365 дней);
- `age` - Возраст;
- `gender_segment` - Пол;
- `os_name` - Тип операционной системы;
- `cpe_type_name` - Тип устройства;
- `country` - Страна проживания;
- `city` - Город проживания;
- `age_segment` - Возрастной сегмент;
- `traffic_segment` - Сегмент по объёму потребляемого трафика;
- `lifetime_segment` - Сегмент по количеству дней «жизни»;
- `nps_score` - Оценка клиента в NPS-опросе;
- `nps_group` - Поле хранит информацию о том, к какой группе относится оценка клиента в опросе.

Выгрузим базу данных при помощи SQL-запросов:

In [3]:
query = """
SELECT u.user_id,
       ABS(u.lt_day) lt_day,
       CASE
           WHEN lt_day <= 365 THEN 'New'
           ELSE 'Old'
       END is_new,
       u.age,
       u.os_name,
       u.cpe_type_name,
       u.nps_score,
       l.country,
       l.city,
       CASE
           WHEN gender_segment == 1 THEN 'Женщина'
           WHEN gender_segment == 0 THEN 'Мужчина'
           ELSE 'Неизвестно'
       END gender_segment,
       SUBSTR(a_s.title, 4, 5) age_segment,
       SUBSTR(t_s.title, 4, 6) traffic_segment,
       SUBSTR(l_s.title, 4, 5) lifetime_segment,
       CASE
           WHEN nps_score >= 9 THEN 'Сторонники'
           WHEN nps_score >= 7 AND u.nps_score <= 8 THEN 'Нейтралы'
           WHEN nps_score <= 6 THEN 'Критики'
       END nps_group
       
FROM location as l
LEFT JOIN user AS u ON l.location_id = u.location_id
LEFT JOIN age_segment AS a_s ON u.age_gr_id = a_s.age_gr_id
LEFT JOIN traffic_segment AS t_s ON u.tr_gr_id = t_s.tr_gr_id
LEFT JOIN lifetime_segment AS l_s ON u.lt_gr_id = l_s.lt_gr_id
;
"""

Создаём датафрейм по данным запроса:

In [6]:
df = pd.read_sql(query, engine)

# Проверим, что датафрейм корректно отображается:
df.head(5)

,user_id,lt_day,is_new,age,os_name,cpe_type_name,nps_score,country,city,gender_segment,age_segment,traffic_segment,lifetime_segment,nps_group
0,A0CMN5,2729,Old,27.0,ANDROID,SMARTPHONE,6,Россия,Архангельск,Мужчина,25-34,60-65,36+,Критики
1,A0CYVK,478,Old,48.0,ANDROID,SMARTPHONE,7,Россия,Архангельск,Женщина,45-54,10-15,13-24,Нейтралы
2,A0HNQU,413,Old,42.0,ANDROID,SMARTPHONE,9,Россия,Архангельск,Женщина,35-44,20-25,13-24,Сторонники
3,A0QZ38,1031,Old,30.0,ANDROID,SMARTPHONE,8,Россия,Архангельск,Мужчина,25-34,1-5,25-36,Нейтралы
4,A0RZLZ,604,Old,55.0,ANDROID,SMARTPHONE,10,Россия,Архангельск,Мужчина,55-64,5-10,13-24,Сторонники


Получившуюся таблицу сохраним как CSV-файл:

In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

### Дашборд

Дальнейший анализ и его визуализация проходили в Tableau. Был построен дашборд, предоставляющий информацию о текущем уровне NPS среди клиентов и показывающий, как этот уровень меняется в зависимости от пользовательских признаков. Дашборд можно открыть по ссылке ниже:

[Дашборд в Tableau](https://public.tableau.com/app/profile/nadiia4738/viz/_16742343909190/Dashboard1?publish=yes)

По данным из дашборда удалось достичь цели исследования, ответив на следующие вопросы:

- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников?

### Результаты с выводами исследования можно прочитать в презентации по ссылке ниже:

[Презентация итогов исследования](https://drive.google.com/file/d/1OD9Tu_uPrKl5REvBMjSLmPxK0QzJYnOb/view?usp=sharing)